# TensorFlow - Keras: Clasificación de textos con Redes Neuronales  (Perceptron Multicapa)


* En este notebook vamos a ver como crear un modelo que clasifique las frases según su polaridad (Positiva o Negativa) con una red neuronal, en particular con un ***Perceptrón Multicapa***.

* En este notebook vamos a tener dos conjuntos de frases clasificadas según su polaridad:
    1. **train**: Conjunto de frases clasificadas para entrenar la red.
    3. **Test**: Conjunto de frases de test a predecir por la red.
    
    
* Los pasos que vamos a dar son los siguientes:

    1. Normalización de las frases.
    2. Bolsa de palabras y codificación del target
    3. Definición de la Red Neuronal
    4. Entrenamiento de la Red Neuronal
    5. Evaluación del modelo

<hr>


# Frases a clasificar

In [19]:
train = [('I love this car', 'positive'),
         ('This view is amazing', 'positive'),
         ('I feel great this morning', 'positive'),
         ('I am so excited about the concert', 'positive'),
         ('He is my best friend', 'positive'),
         ('Going well', 'positive'),
         ('Thank you', 'positive'),
         ('Hope you are doing well', 'positive'),
         ('I am very happy', 'positive'),
         ('Good for you', 'positive'),
         ('It is all good. I know about it and I accept it.', 'positive'),
         ('This is really good!', 'positive'),
         ('Tomorrow is going to be fun.', 'positive'),
         ('These are great apples today.', 'positive'),
         ('How about them apples? Thomas is a happy boy.', 'positive'),
         ('I love this sandwich.', 'positive'),
         ('This is an amazing place!', 'positive'),
         ('I feel very good about these beers.', 'positive'),
         ('This is my best work.', 'positive'),
         ('What an awesome view', 'positive'),
         ('I do not like this car', 'negative'),
         ('This view is horrible', 'negative'),
         ('I feel tired this morning', 'negative'),
         ('I am not looking forward to the concert', 'negative'),
         ('He is my enemy', 'negative'),
         ('I am a bad boy', 'negative'),
         ('This is not good', 'negative'),
         ('I am bothered by this', 'negative'),
         ('I am not connected with this', 'negative'),
         ('Sadistic creep you ass. Die.', 'negative'),
         ('All sorts of crazy and scary as hell.', 'negative'),
         ('Not his emails, no.', 'negative'),
         ('His father is dead. Returned obviously.', 'negative'),
         ('He has a bomb.', 'negative'),
         ('Too fast to be on foot. We cannot catch them.', 'negative'),
         ('I do not like this restaurant', 'negative'),
         ('I am tired of this stuff.', 'negative'),
         ("I can't deal with this", 'negative'),
         ('He is my sworn enemy!', 'negative'),
         ('My boss is horrible.', 'negative')]

test = [('I feel happy this morning', 'positive'),
        ('Larry is my friend', 'positive'),
        ('I do not like that man', 'negative'),
        ('My house is not great', 'negative'),
        ('Your song is annoying', 'negative'),
        ('The beer was good.', 'positive'),
        ('I do not enjoy my job', 'negative'),
        ("I feel amazing!", 'positive'),
        ('Gary is a friend of mine.', 'positive'),
        ("I can't believe I'm doing this.", 'negative')]

<hr>


## Normalización

* En primer lugar vamos a pasar a normalizar las frases. Para ello realizaremos lo siguiente:

    - Eliminamos los signos de puntuación
    - Pasamos el texto a minúsculas
    - Eliminamos las Stop-Words
    - Eliminamos las palabras con menos de 3 caracteres
    
* Nos creamos una función que realice este procesamiento para las frases dadas.


***NOTA***: *Para este ejemplo en particular se hace una normalización muy básica pero suficiente para realizar este ejemplo con caracter didáctico*.

In [20]:
import warnings
warnings.filterwarnings("ignore")
import re

from nltk.corpus import stopwords
import nltk
nltk.download('all')

def normalize(sentenses):
    """normalizamos la lista de frases"""
    sen = []
    for (words, sentiment) in sentenses:
        words_filtered = []
        for word in words.split():
            # Eliminamos signos de puntuación y lo pasamos a minusculas
            word = re.sub(r'[^\w\s]', '', word).lower()
            # Filtramos stop words y las palabras con menos de 3 caracteres
            if len(word) > 2 and word not in stopwords.words():
                words_filtered.append(word)
        sen.append(words_filtered)
    return sen

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

* Pasamos a normalizar los conjuntos de entrenamiento y test.


* Obtenemos en una lista el target de los conjunto de entrenamiento y test.

In [21]:
X_train = normalize(train)
y_train = [sentiment for (words, sentiment) in train]
X_test = normalize(test)
y_test = [sentiment for (words, sentiment) in test]

<hr>


# Bolsa de palabras y codificación del target

* Cuando trabajamos con redes neuronales para la clasificación de textos es necesario tener:

    1. ***Input***: Un vector por documento en el que en cada posición del vertor represente el peso que tiene la palabra en ese documento bien sea por frecuencia, por aparición de la palabra en el documento (One-Hot-Encode) o por su TF-IDF.
<span></span><br><br>    
    2. ***Output***: Al resolver la red neuronal un problema de clasificación de textos, la salida de la red tiene que ser un valor numérico, por lo que el target hay que codificalo en valores numéricos. En este ejemplo al ser una clasificación binaria (positivo o negativo) codificaremos estos dos valores en '0' y '1'.
    
    
* A continuación pasamos a crear la bolsa de palabras.


* Para realizar esta labor vamos a usar la clase '***Tokenizer***' de Tensorflow: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text


* Lo que vamos a hacer es crear una ***bolsa de palabras de frecuencias*** cogiendo las 'N' palabras más frecuentes del Corpus. En este caso el Corpus tiene 63 palabras por lo que vamos a seleccionar todas las palabras.


* Es muy importante saber la ***dimensión (longitud) del vector de palabras*** que representará al documento ya que esa dimensión será el ***número de neuronas de entrada*** (+1) que tendrá nuestra red neuronal.


* La bolsa de palabras la vamos a crear como una matriz llamando a la función '***text_to_matrix***'

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer


num_words = 64

# Creamos un objeto de la clase Tokenizer indicandole el número de palabras
tokenizer = Tokenizer(num_words=num_words)

# Calculamos la bolsa de palabras (generamos el vocabulario)
tokenizer.fit_on_texts(X_train)

# Creamos una matriz de bolsa de palabras donde:
#     Fila: Representa a un documento
#     Columna: Representa a una palabra
X_matrix_train = tokenizer.texts_to_matrix(X_train, mode='count')
X_matrix_test = tokenizer.texts_to_matrix(X_test, mode='count')

print('Dimensión de la matriz de entrenamiento: {dim}\n'.format(dim=X_matrix_train.shape))

print('Bolsa de palabras de los datos de entrenamiento:\n{bolsa}\n'.format(bolsa=tokenizer.word_counts))

print('Índice de las palabras en la Matriz:\n{index}\n'.format(index=tokenizer.word_index))

print('Matriz de entrenamiento:\n{matrix}\n'.format(matrix=X_matrix_train))

Dimensión de la matriz de entrenamiento: (40, 64)

Bolsa de palabras de los datos de entrenamiento:
OrderedDict([('love', 2), ('car', 2), ('view', 3), ('amazing', 2), ('feel', 3), ('great', 2), ('morning', 2), ('excited', 1), ('concert', 2), ('friend', 1), ('hope', 1), ('happy', 2), ('accept', 1), ('tomorrow', 1), ('fun', 1), ('apples', 2), ('today', 1), ('thomas', 1), ('boy', 2), ('sandwich', 1), ('place', 1), ('beers', 1), ('work', 1), ('awesome', 1), ('horrible', 2), ('tired', 2), ('forward', 1), ('enemy', 2), ('bothered', 1), ('connected', 1), ('sadistic', 1), ('creep', 1), ('ass', 1), ('sorts', 1), ('crazy', 1), ('scary', 1), ('hell', 1), ('emails', 1), ('father', 1), ('dead', 1), ('returned', 1), ('bomb', 1), ('fast', 1), ('foot', 1), ('catch', 1), ('restaurant', 1), ('stuff', 1), ('deal', 1), ('sworn', 1), ('boss', 1)])

Índice de las palabras en la Matriz:
{'view': 1, 'feel': 2, 'love': 3, 'car': 4, 'amazing': 5, 'great': 6, 'morning': 7, 'concert': 8, 'happy': 9, 'apples': 10,

* Lo siguiente que vamos a hacer en codificar la salida
    - Negativo: 0
    - Positivo: 1
    

* Esto lo vamos a hacer con la clase '***LabelEncoder***' de scikit: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

In [23]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)
y_train

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

<hr>


# Definición de la Red Neuronal


* Para clasificar los textos en positivo o negativo vamos a crear una red con la siguiente arquitectura:
<span></span><br><br>
    - ***Capa 1***: Capa de entrada con 64 Neuronas (una por palabra del corpus)
<span></span><br><br>
    - ***Capa 2***: 20 Neuronas con una función de activación 'relu'
<span></span><br><br>
    - ***Capa 3***: 5 Neuronas con una función de activación 'relu'
<span></span><br><br>
    - ***Capa 4***: Capa de salida con 1 neurona y una función de activación 'sigmoidal'
    
    
* Para las capas 2 y 3 vamos a poner un dropout del 10% para que nuestra red no se sobreajuste y generalize mejor.
    
    
* Por último utilizaremos:
<span></span><br><br>
    - ***Función de perdida: 'binary_crossentropy'*** (Funciones de Perdida: https://keras.io/losses/)
<span></span><br><br>
    - ***Optimizador: 'adam'*** (Optimizadores: https://keras.io/optimizers/)
<span></span><br><br>    
    - Metricas: en cada epoch pediremos que nos muestre el accuracy (Métricas: https://keras.io/metrics/)
    


In [24]:
# Se fuerza a usar la CPU y no la GPU en caso de tener GPU en el ordenador
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [25]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import metrics

np.random.seed(2)
model = Sequential()
model.add(Dense(20, activation='relu', input_dim=num_words))
model.add(Dropout(0.1))
model.add(Dense(5, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


* Una vez creada la red mostramos su arquitectura:
    - 64 Neuronas de entrada
    - Capa 2: 20 neuronas
    - Conexiones "capa de entrada -> Capa 2" = (64*20) + 20 = 1300 Conexiones
    - Capa 3: 5 neuronas
    - Conexiones "Capa 2 -> Capa 3" = (20*5) + 5 = 105 Conexiones
    - Capa de salida: 1 Neurona
    - Conexiones "Capa 3 -> Capa de salida" = (5*1) + 1 = 6 Conexiones

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 20)                1300      
                                                                 
 dropout_2 (Dropout)         (None, 20)                0         
                                                                 
 dense_4 (Dense)             (None, 5)                 105       
                                                                 
 dropout_3 (Dropout)         (None, 5)                 0         
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1411 (5.51 KB)
Trainable params: 1411 (5.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


<hr>


# Entrenamos la red


* Una vez definida la Red Nuronal y compilada ya podemos entrenarla (ajustar los pesos de las conexiones).


* Para entrenar la Red Nuronal (model) vamos a llamar al método '***fit()***' que recibe los siguientes parámetros:
<span></span><br><br>
    - ***datos de entrada*** (X_matrix_train)
<span></span><br><br>
    - ***target*** (y_train)
<span></span><br><br>
    - ***batch_size***: Número de muestras por epoch que utilizará para actualizar los pesos
<span></span><br><br>
    - ***epochs***: Número de veces que recorrerá el dataset para entrenar la red
<span></span><br><br>
    - ***verbose***: 3 tipos de verbosidad mientras entrena la red (0,1,2)
<span></span><br><br>
    - ***validation_split***: Porcentaje de los datos de entrenamiento que utiliza para validar el modelo en cada epoch.
<span></span><br><br>
    
    
* Como lo que estamos haciendo es un ejemplo didáctico vamos a poner los siguientes valores:
<span></span><br><br>
    - *batch_size = 1* : Como tenemos pocos datos, podemos ajustar los pesos de la Red Neuronal cada vez que se calcule una muestra.
        + batch_size con valores muy pequeños: En teoria la red neuronal estaría muy bien entrenada pero tardaría mucho tiempo en entrenarse.
        + batch_size con valores muy grandes: En teoria la red neuronal estaría peor entrenada que con valores pequeños pero tardaría menos tiempo en ejecutarse.
<span></span><br><br>
    - *epochs = 20* : Como es un dataset pequeño no es necesario entrenar la red muchas veces por eso ponemos un valor relativamente pequeño.
<span></span><br><br>
    - *validation_split = 0.1* : Al ser un dataset con 40 frases, elegiremos en cada pasada solo 4 frases para evaluarlas.

In [27]:
model.fit(X_matrix_train, y_train, batch_size=1, epochs=20, verbose=2, validation_split=0.1)

Epoch 1/20
36/36 - 2s - loss: 0.7339 - accuracy: 0.4722 - val_loss: 0.5497 - val_accuracy: 1.0000 - 2s/epoch - 54ms/step
Epoch 2/20
36/36 - 0s - loss: 0.6894 - accuracy: 0.5556 - val_loss: 0.5606 - val_accuracy: 1.0000 - 138ms/epoch - 4ms/step
Epoch 3/20
36/36 - 0s - loss: 0.6818 - accuracy: 0.5833 - val_loss: 0.5714 - val_accuracy: 1.0000 - 149ms/epoch - 4ms/step
Epoch 4/20
36/36 - 0s - loss: 0.6535 - accuracy: 0.6389 - val_loss: 0.5571 - val_accuracy: 1.0000 - 126ms/epoch - 4ms/step
Epoch 5/20
36/36 - 0s - loss: 0.6513 - accuracy: 0.5833 - val_loss: 0.5540 - val_accuracy: 1.0000 - 135ms/epoch - 4ms/step
Epoch 6/20
36/36 - 0s - loss: 0.6527 - accuracy: 0.6389 - val_loss: 0.5612 - val_accuracy: 1.0000 - 140ms/epoch - 4ms/step
Epoch 7/20
36/36 - 0s - loss: 0.6328 - accuracy: 0.6389 - val_loss: 0.5505 - val_accuracy: 1.0000 - 133ms/epoch - 4ms/step
Epoch 8/20
36/36 - 0s - loss: 0.6212 - accuracy: 0.6667 - val_loss: 0.5450 - val_accuracy: 0.7500 - 104ms/epoch - 3ms/step
Epoch 9/20
36/36 -

<hr>


# Evaluamos el modelo


* Al tratarse de un ejemplo didáctico vamos a medir solamente el accuracy.


* Para ello utilizaremos el método '***evaluate()***' que pasandole los datos de entrenamiento (datos y target), predice los datos y nos calcula el accuracy entre otras métricas.

In [28]:
# Evaluación del modelo con los mismos datos de entrenamiento
scores = model.evaluate(X_matrix_test, y_test)
print("Porcentaje acierto: %s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))


1/1 [==============================] - 0s 140ms/step - loss: 0.7084 - accuracy: 0.5000
Porcentaje acierto: accuracy: 50.00%


* Llamando al método '***predict()***' y pasandole unos datos de entrada, nos devuelve el valor de la salida.

In [29]:
predict = model.predict(X_matrix_test)
print(predict)

1/1 [==============================] - 0s 65ms/step
[[0.5876003 ]
 [0.58716613]
 [0.5700914 ]
 [0.6286877 ]
 [0.5700914 ]
 [0.5700914 ]
 [0.5700914 ]
 [0.57252645]
 [0.58716613]
 [0.5700914 ]]


* Como lo que nos interesa en un problema de clasificación es la categoria en que nos clasifica el documento y no el valor de salida de la red neuronal, podemos obtener la categoria de cada predicción llamando al método '***predict_classes()***', pasandole el/los dato/s de entrada.


* Hay que tener cuidado ya que nos va a dar un número entero (0 o 1) ya que hemos realizado el LabelEncoder previamente y tenemos que hacer el "encodeado inverso":

In [30]:
# Predecimos los resultados del conjunto de test
y_predict = np.where(model.predict(X_matrix_test) > 0.5, 1, 0)
for index, prediction in enumerate(y_predict):
    text = test[index][0]
    y_true = test[index][1]
    y_predict = encoder.inverse_transform(prediction)
    print('{i} - {text} \n\t Real: {y_true} - Predicción: {y_predict} - Acierto: {acierto}'
          .format(i=index, text=text, y_true=y_true, y_predict=y_predict[0], acierto = y_true==y_predict[0]))

1/1 [==============================] - 0s 23ms/step
0 - I feel happy this morning 
	 Real: positive - Predicción: positive - Acierto: True
1 - Larry is my friend 
	 Real: positive - Predicción: positive - Acierto: True
2 - I do not like that man 
	 Real: negative - Predicción: positive - Acierto: False
3 - My house is not great 
	 Real: negative - Predicción: positive - Acierto: False
4 - Your song is annoying 
	 Real: negative - Predicción: positive - Acierto: False
5 - The beer was good. 
	 Real: positive - Predicción: positive - Acierto: True
6 - I do not enjoy my job 
	 Real: negative - Predicción: positive - Acierto: False
7 - I feel amazing! 
	 Real: positive - Predicción: positive - Acierto: True
8 - Gary is a friend of mine. 
	 Real: positive - Predicción: positive - Acierto: True
9 - I can't believe I'm doing this. 
	 Real: negative - Predicción: positive - Acierto: False


<hr>


# Perceptrón Multicapa (MLP) -


* El Perceptrón Multicapa (Multilayer Perceptron- MLP) es una Red Neuronal Artificial que tiene como objetivo la resolución de problemas de clasificación o regresión que no son linealmente separables.


* El MLP esta formado por una serie de capas compuesta por neuronas, estando cada neurona de una capa conectada con todas las neuronas de la capa posterior. Las capas de una red neuronal las podemos dividir en 3 tipos que son:
<span></span><br><br>
    - ***Capa de entrada***: Esta formada por tantas neuronas como variables tengan los elementos de entrada. Las Neuronas de la capa de entrada no realizan ningún tipo de procesamiento.
<span></span><br><br>
    - ***Capas ocultas***: Esta compuesta por una serie de neuronas cuyas entradas provienen de las salidas de las neuronas de la capa anterior y sus salidas sirven como entrada a cada una de las neuronas de la capas posterior.
<span></span><br><br>
    - ***Capas de salida***: Esta compuesta por una o más neuronas, y los valores de estas neuronas corresponden con la salida de la red.
    
    
* Un ejemplo de arquitectura de una red neuronal con una capa de entrada de 2 neuronas, 3 capas ocultas de 5, 3 y 2 neuronas respectivamente y una capa de salida de una neurona, sería la siguiente:


## Entrenar una Red Neuronal


* De manera "general", pera crear y entrenar una Red Neuronal (un Perceptrón Multicapa) tenemos que seguir los siguientes pasos:
<span></span><br>
    1. ***Recopilar conjunto de datos*** (Cuantos más datos mejor).
<span></span><br><br>
    2. ***Diseñar una función de perdida*** (loss function) apropiada para el problema; por ejemplo:
        + MSE para problemas de regresión.
        + Cross Entropy para problemas de clasificación (Clasificación binaria: “binary crossentropy” y Clasificación Múltiple: “categorical_crossentropy”).
<span></span><br><br>
    3. ***Definir la arquitectura de la Red Neuronal*** y sus hiperparámetros.
        + Número de Capas y Neuronas por Capa.
        + Funciones de Activación.
        + Hiperparámetros: Learning Rate, Regularization Rate, Epochs, Batch Size, etc.
<span></span><br><br>
    4. ***Aplicar un algoritmo de optimización*** para minimizar la función de pérdida para que ajuste los pesos de la red:
        + Stochastic Gradient Descent (SGD)
        + RMSProp
        + Adam
        + AdaGrad
        + AdaDelta
        + AdaMax


## Dropout


* El Dropout es un método que se utilizada para la regularización y tiene como objetivo reducir el overfiting.


* Consiste en ***perturbar la red*** en cada pasada de entrenamiento (feed-forward y backpropagation), ***eliminando al azar algunas de las unidades de cada capa***.


* El objeto es que al introducir ruido en el proceso de entrenamiento evitamos el overfiting, pues en cada paso de la iteración estamos limitando el número de unidades que la red puede usar para ajustar las respuestas.




## Hiperparámetros de la Red

### Epochs


* Los Epochs (las épocas en Castellano) es un hiperparámetro que indica el ***número de veces que la Red Neuronal aprenderá de todas las observaciones de Dataset***.


* Por ejemplo, si tenemos un Dataset con 200K observaciones y le indicamos a la red que realice 50 epochs, esto significa que la Red Neuronal leerá y aprenderá 50 veces las 200K observaciones del Dataset; es decir, que leerá 10.000K observaciones (200k x 50).


### Batch Size


* El Batch Size es un hiperparámetro que indica el ***número de observaciones que tiene que leer la Red Neuronal antes de actualizar el modelo*** (los pesos de la Red Neuronal).


* Por ejemplo con un Batch Size de 100 lo que haremos será calcular la salida para 100 Observaciones y calcular sus errores en función de la predicción que realice la Red. Posteriormente se calcula el error medio de las 100 observaciones y se actualizan los pesos de la Red Neuronal.

    + Un Batch Size pequeño:  la Red Neuronal aprenda muy bien pero tardará mucho tiempo en calcular el modelo.
    
    + Barch Size grande: la Red Neuronal no aprenda tan bien pero tardará menos tiempo en calcular el modelo.


<hr>



# TensorFlow - Keras -

* TensorFlow es una librería de Deep Learning en Python desarrollada por google.


* Keras es una librería en Python desarrollada por François Chollet (ingeniero de Google) que en esencia es un wrapper sobre TensorFlow (version 1), Theano y CNTK: https://github.com/keras-team/keras


* TensorFlow (V1), Theano y CNTK son librería muy potentes pero tienen el “problema” que de son muy difíciles de utilizar, por lo que Keras empezó a desarrollarse por Chollet con el objetivo de crear una librería sencilla de utilizar para temas de Deep Learning.


* TensorFlow (con el objetivo de simplificar y democratizar el uso de su librería) se integro con el API de Keras, para hacer un uso más sencillo de su librería. Esto es así a partir de la versión 2 de TensorFlow.

* Los principios en los que se basaba Keras (y de ahi la integración con TensorFlow) son los siguientes:
<span></span><br><br>
    - ***Modularidad***: Una red neuronal se entiende como una secuencia o grafo de capas de neuronas.
<span></span><br><br>
    - ***Minimalismo***: La librería proporciona lo justo y necesario para crear y entrenar redes maximizando la legibilidad del código.
<span></span><br><br>
    - ***Extensibilidad***: Los nuevos componentes tienen que ser fáciles de añadirse a Keras.
<span></span><br><br>
    - ***Python***: Keras es Python nativo.



## Como Construir modelos en TensorFlow - Keras


* Lo que a continuación se cuenta se hace de manera muy genérica pero podemos definir los pasos que se deben de dar para contruir una red neuronal:
<span></span><br><br>
    1. ***Definir el modelo***: crear el ***Sequential*** model y añadir las capas con su configuración (https://www.tensorflow.org/api_docs/python/tf/keras/Sequential).
<span></span><br><br>
    2. ***Compilar el modelo***: definir o especificar la función de pérdida y llamar al método ***compile()***.
<span></span><br><br>
    3. ***Ajustar (fit) el modelo***: entrenar el modelo con los datos de entrenamiento llamando al método ***fit()***.
<span></span><br><br>
    4. ***Realizar predicciones***: usando el modelo se pueden realizar predicciones con nuevos datos llamando a los métodos ***evaluate()*** o ***predict()***.
